In [1]:
# !pip install datasets
# !pip install torchmetrics
# !pip install pytz
# !pip install persiantools
# !pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.0/212.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
     ━━━━━━━━━━━━━━━━

# Mount Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# %cd /content/drive/MyDrive

/content/drive/MyDrive


# Configuration

In [1]:
config = {}
config['root_path'] = '/home/user01/'
config['series_ID'] = 102
config['series_desc'] = '''
third week - 29 mehr 1401 - fixed many many things including the targeted backdoor attack, success measures, etc.
'''
config['log_path'] = config['root_path']+'experiments/MNIST/SVM/'
config['log'] = {}

In [2]:
config['poisoning_rate'] = 0.0
# config['num_clean_examples'] = 0
config['learning_rate'] = 0.01
config['batch_size'] = 32
config['num_epochs'] = 2

In [3]:
config['log']['model'] = 'SVM'
config['log']['dataset'] = 'MNIST (0-1)'
config['log']['task'] = 'binary classification'
config['log']['pytorch_seed'] = 50
config['log']['numpy_seed'] = 50
config['log']['attack'] = 'no'

In [4]:
config['log_path'] += (str(config['series_ID']) + '-' + config['log']['attack'] + '-' + config['log']['dataset'] + '-' + str(int(config['poisoning_rate']*100)) + '.json').lower().replace(' ', '')

In [5]:
config['log_path']

'/home/user01/experiments/MNIST/SVM/102-no-mnist(0-1)-0.json'

In [6]:
### WE NEED THIS TO IMPORT THE NECESSARY LIBRARIES ###
import sys
sys.path.append(config['root_path'])

In [7]:
import datascience, poisoning, report
from datascience.data import CIFAR10, MNIST, IMDB, BOSTON
from datascience.general import train_dev_test_split, join_np_arrays, describe_dataset
from poisoning.process import attacker, defender, SVM_KKT_attacker, SVM_influence_attacker, LR_influence_attacker, label_flip_attacker, targeted_backdoor_attacker_img, targeted_backdoor_attacker_txt
from poisoning.eval import attack_success_rate, benign_accuracy, test_accuracy
from report.log import JSONLogger, TextLogger, tehran_datetime
from temporary.functions import _reload
import numpy as np
import torch
from torchmetrics import HingeLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd.functional import hessian, jacobian
from torch.autograd import grad
from torch.nn.utils import _stateless
from datetime import datetime
import pytz
from persiantools.jdatetime import JalaliDate

2022-11-17 20:53:02.241328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 20:53:02.410408: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 20:53:03.084703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: usr/local/cuda-11.8/lib64
2022-11-17 20:53:03.084843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [8]:
_reload(poisoning.process)
_reload(poisoning.eval)
_reload(datascience.data)
_reload(datascience.general)
_reload(report.log)

<module 'report.log' from '/home/user01/report/log.py'>

In [12]:
torch.manual_seed(config['log']['pytorch_seed'])

# Loading a Dataset

In [13]:
dataset = MNIST()
dataset.select_labels([0,1], phase='train')
dataset.select_labels([0,1], phase='test')
dataset.flatten()

In [ ]:
# dataset.separate_examples(config['num_clean_examples'], desired_seed = config['log']['numpy_seed'])

In [ ]:
# dataset.change_labels({0:-1})

In [14]:
config['data-train'] = describe_dataset(dataset.x_train, dataset.y_train, 'training dataset')

In [15]:
config['data-test'] = describe_dataset(dataset.x_test, dataset.y_test, 'testing dataset')

# Prepare the Data

In [41]:
class MyVectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]).to(device), torch.Tensor(self.labels[idx]).to(device)

In [42]:
train_dataset = MyVectorDataset(dataset.x_train, dataset.y_train)
test_dataset = MyVectorDataset(dataset.x_test, dataset.y_test)
# clean_dataset = MyVectorDataset(dataset.x_train_clean, dataset.y_train_clean)

In [43]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
# clean_dataloader = DataLoader(clean_dataset, batch_size=config['batch_size'], shuffle=True)

# Train a Model

In [44]:
class SVM(torch.nn.Module):
    def __init__(self):
        super(SVM, self).__init__() 
        self.linear = torch.nn.Linear(in_features=784, out_features=1, bias=True)
    def forward(self, x):
        output = self.linear(x)
        return output

In [45]:
def output_to_label(out):
    if out >= 0:
      return 1
    else:
      return 0

In [46]:
def make_prediction(model, x_arr):
  outs = list(model(torch.Tensor(x_arr)).squeeze().detach().numpy())
  labels = [output_to_label(out) for out in outs]
  return np.array(labels)

In [48]:
model = SVM()
model = model.to(device)
loss_fn = HingeLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)

In [49]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")

In [53]:
def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.cpu().numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.cpu().numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    config['log']['accuracy_'+name] = (100*correct)
    config['log']['loss_'+name] = sum_test_loss
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [51]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch_num)

 Epoch 2 - loss: 0.000000  [ 9600/12665] 

In [54]:
test_loop(train_dataloader, model, loss_fn, config['num_epochs'], 'Train')

 Epoch 2 - Train Error: Accuracy: 99.9%, Avg loss: 0.022522 

In [55]:
test_loop(test_dataloader, model, loss_fn, config['num_epochs'], 'Test')

 Epoch 2 - Test Error: Accuracy: 100.0%, Avg loss: 0.011732 

In [28]:
y_train_pred = make_prediction(model, dataset.x_train)
y_test_pred = make_prediction(model, dataset.x_test)

In [59]:
config['datetime'] = tehran_datetime()

In [60]:
config

{'series_ID': 102,
 'series_desc': '\nthird week - 29 mehr 1401 - fixed many many things including the targeted backdoor attack, success measures, etc.\n',
 'log_path': '/home/user01/experiments/MNIST/SVM/102-no-mnist(0-1)-0.json',
 'log': {'model': 'SVM',
  'dataset': 'MNIST (0-1)',
  'task': 'binary classification',
  'pytorch_seed': 50,
  'numpy_seed': 50,
  'attack': 'no',
  'accuracy_Train': 99.87366758784052,
  'loss_Train': 0.02252219046204635,
  'accuracy_Test': 99.95271867612293,
  'loss_Test': 0.011732068974921043},
 'poisoning_rate': 0.0,
 'learning_rate': 0.01,
 'batch_size': 32,
 'num_epochs': 2,
 'data-train': {'name': 'training dataset',
  'num_samples': 12665,
  'num_features': (784,),
  'class_count': {0: 5923, 1: 6742}},
 'data-test': {'name': 'testing dataset',
  'num_samples': 2115,
  'num_features': (784,),
  'class_count': {0: 980, 1: 1135}},
 'datetime': '1401-08-26 00:47'}

In [36]:
logger = JSONLogger(config['log_path'], config)
logger.log()